In [1]:
import numpy as np
from skimage import io
from os.path import expanduser
from tqdm import tqdm
HOME = expanduser("~")
import os, sys
import cv2
import numpy as np
import SimpleITK as sitk
from collections import OrderedDict
from shutil import copyfile
import subprocess
import pickle
%load_ext autoreload
%autoreload 2

In [2]:
PATH = '/home/eddyod/programming/pipeline_utility'
sys.path.append(PATH)
from utilities.sqlcontroller import SqlController
from utilities.file_location import FileLocationManager
from utilities.utilities_registration import create_warp_transforms, register
from utilities.alignment_utility import SCALING_FACTOR

Connecting dklab@192.168.1.12:3306


In [3]:
animal = 'DK39'
DIR = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps'
INPUT = os.path.join(DIR, 'CH1', 'thumbnail_cleaned')
OUTPUT = os.path.join(DIR, 'CH1', 'thumbnail_aligned')
STORAGE = os.path.join(DIR, 'elastix')

In [4]:
rotation_storage = os.path.join(STORAGE, 'rotations.pickle')
rotations = pickle.load( open( rotation_storage, "rb" ) )

#translation_storage = os.path.join(STORAGE, 'translations.pickle')
#translations = pickle.load( open( translation_storage, "rb" ) )

In [5]:
transformation_to_previous_section = OrderedDict()
for rf, R in rotations.items():
    transformation_to_previous_section[rf] = R

files = sorted(os.listdir(INPUT))

In [6]:
anchor_index = len(files) // 2 # middle section of the brain
transformation_to_anchor_section = {}
# Converts every transformation
for moving_index in range(len(files)):
    if moving_index == anchor_index:
        transformation_to_anchor_section[files[moving_index]] = np.eye(3)
    elif moving_index < anchor_index:
        T_composed = np.eye(3)
        for i in range(anchor_index, moving_index, -1):
            T_composed = np.dot(np.linalg.inv(transformation_to_previous_section[files[i]]), T_composed)
        transformation_to_anchor_section[files[moving_index]] = T_composed
    else:
        T_composed = np.eye(3)
        for i in range(anchor_index + 1, moving_index + 1):
            T_composed = np.dot(transformation_to_previous_section[files[i]], T_composed)
        transformation_to_anchor_section[files[moving_index]] = T_composed


#### Set the resolution and create a dictionary of transforms

In [7]:
resolution = 'thumbnail'
warp_transforms = create_warp_transforms(animal, transformation_to_anchor_section, 'thumbnail', resolution)
sqlController = SqlController(animal)
width = sqlController.scan_run.width
height = sqlController.scan_run.height
max_width = int(width * SCALING_FACTOR)
max_height = int(height * SCALING_FACTOR)
bgcolor = 'black' # this should be black, but white lets you see the rotation and shift

##### This gets the dictionary of transforms and passes those parameters to imagemagick's convert tool to rotate, shift and crop the image

In [8]:
#OUTPUT = "setme to some place where you can write files"
ordered_transforms = OrderedDict(sorted(warp_transforms.items()))
for file, arr in tqdm(ordered_transforms.items()):
    T = np.linalg.inv(arr)
    sx = T[0, 0]
    sy = T[1, 1]
    rx = T[1, 0]
    ry = T[0, 1]
    tx = T[0, 2]
    ty = T[1, 2]
    # sx, rx, ry, sy, tx, ty
    op_str = f" +distort AffineProjection '{sx},{rx},{ry},{sy},{tx},{ty}'"
    op_str += f' -crop {max_width}x{max_height}+0.0+0.0!'
    input_fp = os.path.join(INPUT, file)
    output_fp = os.path.join(OUTPUT, file)
    if os.path.exists(output_fp):
        continue

    cmd = f"convert {input_fp} -define white-point=0x0 +repage -virtual-pixel background -background {bgcolor} {op_str} -flatten -compress lzw {output_fp}"
    subprocess.run(cmd, shell=True)


100%|██████████| 465/465 [01:44<00:00,  4.47it/s]
